# Day 21: Dirac Dice

In [1]:
example = """Player 1 starting position: 4
Player 2 starting position: 8"""

In [2]:
def parse(input):
    """Parses player 1 and 2 starting positions from input."""
    return [int(line.split(': ')[1]) for line in input.splitlines()]
parse(example)

[4, 8]

Deterministic die cycles from 1 to 100, forever.

In [3]:
import itertools

die = itertools.cycle(range(1, 101))

Cycle between player 1's (0) and player 2's (1) turn.

In [4]:
turns = itertools.cycle(range(2))

Initial player scores are 0.

In [5]:
scores = [0, 0]

Initial player positions come from input.

In [6]:
positions = parse(example)
positions

[4, 8]

Play until game ends.

In [7]:
rolls = 0

# Take turns until a score of at least 1000 is reached.
while all(score < 1000 for score in scores):
    turn = next(turns)

    # Take next three rolls and update position.
    positions[turn] += sum(itertools.islice(die, 3))
    # A little hack because positions start at 1, not 0.
    positions[turn] = ((positions[turn] - 1) % 10) + 1

    # Update score.
    scores[turn] += positions[turn]
    
    # Update roll count.
    rolls +=3

Multiply losing player's score by rolls.

In [8]:
min(scores) * rolls

739785

# Part 1

In [9]:
input = """Player 1 starting position: 1
Player 2 starting position: 6"""
positions = parse(input)
die = itertools.cycle(range(1, 101))
turns = itertools.cycle(range(2))
scores = [0, 0]
rolls = 0

# Take turns until a score of at least 1000 is reached.
while max(scores) < 1000:
    turn = next(turns)

    # Take next three rolls and update position.
    positions[turn] += sum(itertools.islice(die, 3))
    # A little hack because positions start at 1, not 0.
    positions[turn] = ((positions[turn] - 1) % 10) + 1

    # Update score.
    scores[turn] += positions[turn]
    
    # Update roll count.
    rolls +=3

Multiply losing player's score by rolls.

In [10]:
min(scores) * rolls

604998

# Part 2

Each turn there are 3 rolls, and the universe splits into 27 universes.

In [11]:
list(
    itertools.product((1, 2, 3), repeat=3)
)

[(1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 2, 1),
 (1, 2, 2),
 (1, 2, 3),
 (1, 3, 1),
 (1, 3, 2),
 (1, 3, 3),
 (2, 1, 1),
 (2, 1, 2),
 (2, 1, 3),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (2, 3, 1),
 (2, 3, 2),
 (2, 3, 3),
 (3, 1, 1),
 (3, 1, 2),
 (3, 1, 3),
 (3, 2, 1),
 (3, 2, 2),
 (3, 2, 3),
 (3, 3, 1),
 (3, 3, 2),
 (3, 3, 3)]

While there are 27 universe splits, there are only 7 distinct sums of rolls.

In [12]:
len(set(
    sum(rolls) for rolls in itertools.product((1, 2, 3), repeat=3)
))

7

And some sums of rolls are more common than others.

In [13]:
from collections import Counter

rolls_sums_count = Counter(
    sum(rolls) for rolls in itertools.product((1, 2, 3), repeat=3)
)
rolls_sums_count

Counter({3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1})

So while there will be many, many universes, there are only 27 distinct rolls sums for each turn.

Rather than keeping track of all universes separately, we should instead keep track of the number of universes with the same state (position and score).

In [14]:
def play_dirac(initial_positions):
    """Returns states after a player wins in all universes."""
    # Map states (position and score) to count of universes with this state.
    # Initially there are player positions and scores in one universe.
    states = {(tuple(initial_positions), (0, 0)): 1}
    turns = itertools.cycle(range(2))

    # Take turns until all universes have a winner (score of at least 21 is reached).
    while not all(max(scores) >= 21 for positions, scores in states):
        turn = next(turns)
        next_states = {}

        # Iterate over each distinct universe state.
        for (positions, scores), universes in states.items():
            # Skip if there is a winner in this state (any score is at least 21).
            if max(scores) >= 21:
                # If the state already exists, we add to it.
                next_states[(positions, scores)] = next_states.get((positions, scores), 0) + universes
                continue

            # Iterate over each of the 27 rolls sums.
            for rolls_sum, count in rolls_sums_count.items():
                # Convert to lists for index assignment.
                next_positions = list(positions)
                next_scores = list(scores)

                # Update player position.
                next_positions[turn] += rolls_sum
                # A little hack because positions start at 1, not 0.
                next_positions[turn] = ((next_positions[turn] - 1) % 10) + 1

                # Update score.
                next_scores[turn] += next_positions[turn]

                next_state = (tuple(next_positions), tuple(next_scores))

                # Add this state and multiply its universe count by the dice roll count.
                # If the state already exists, we add to it.
                next_states[next_state] = next_states.get(next_state, 0) + universes * count

        states = next_states
    return states

states = play_dirac(parse(example))

Determine total number of universes that each player wins in.

In [15]:
def count_winners(states):
    """Returns counter of universes each player wins in."""
    # 1 if player 1 wins otherwise 2.
    winners = (1 if p1 > p2 else 2 for _, (p1, p2) in states)

    # Sum the winning universes for each player.
    return sum(
        (Counter({player: universes}) for player, universes in zip(winners, states.values())),
        start=Counter()
    )

count_winners(states)

Counter({1: 444356092776315, 2: 341960390180808})

This is correct for the example. Now, for the puzzle input.

In [16]:
states = play_dirac(parse(input))
win_count = count_winners(states)

Find the number of universes the winning player wins in.

In [17]:
win_count.most_common(1)

[(1, 157253621231420)]